# An Empirical Comparison of Supervised and Self-Supervised Learning

In [5]:
import numpy as np
import scipy as sp
from dataclasses import dataclass

from sklearn.svm import SVC
from networks import Encoder, ConvNet
from train import *
from test import *

In [2]:
%load_ext autoreload
%autoreload 2

## Baseline: convolutional neural network

In [3]:
Encoder()

Encoder(
  (conv1): Conv2d(3, 36, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(36, 72, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=18432, out_features=128, bias=True)
)

In [4]:
ConvNet()

ConvNet(
  (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (relu): ReLU()
  (fc1): Linear(in_features=1280, out_features=640, bias=True)
  (fc2): Linear(in_features=640, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
)

## Momentum Contrast

In [10]:
@dataclass
class TrainArgs:
    """Settings for training"""
    batchsize: int = 100
    epochs: int = 50
    out: str = './results'
    no_cuda: bool = False

In [11]:
args = TrainArgs()

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=8)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=8)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Training

In [12]:
# settings
batchsize = args.batchsize
epochs = args.epochs
out_dir = args.out

use_cuda = not args.no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 8, 'pin_memory': True}

In [13]:
# a bunch of random image augmentations
transform = DuplicatedCompose([
    transforms.RandomResizedCrop(32, scale=(0.9, 1.1), ratio=(0.9, 1.1)),
    transforms.RandomRotation(20),
    transforms.RandomGrayscale(p=0.2),
    transforms.RandomApply([
            transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)
        ], p=0.8),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [14]:
# train & test datasets
train_cf10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_cf10 = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_cf10, batch_size=batchsize, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_cf10, batch_size=batchsize, shuffle=True, **kwargs)

Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# initialize encoders and queue
model_q = Encoder().to(device)
model_k = copy.deepcopy(model_q)
optimizer = optim.SGD(model_q.parameters(), lr=0.01, weight_decay=0.0001)

queue = initialize_queue(model_k, device, train_loader)

# start training
for epoch in range(1, epochs + 1):
    train(model_q, model_k, device, train_loader, queue, optimizer, epoch)

os.makedirs(out_dir, exist_ok=True)
torch.save(model_q.state_dict(), os.path.join(out_dir, 'model.pth'))